Autenticación


In [0]:
service_credential = dbutils.secrets.get(scope = "databricks-service-principal-secret-scope", key = "Databricks-server-principal-secret")
spark.conf.set("fs.azure.account.auth.type.jcy203datalake.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.jcy203datalake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.jcy203datalake.dfs.core.windows.net", "dc03afd6-fec8-4dfb-ae3a-73a0af445694")
spark.conf.set("fs.azure.account.oauth2.client.secret.jcy203datalake.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.jcy203datalake.dfs.core.windows.net", "https://login.microsoftonline.com/a3b377f7-39e7-4ca3-bf66-ea94bbdcdbf3/oauth2/token")

In [0]:
from datetime import datetime
from pyspark.sql.functions import explode, col, expr, stack

raw_container = dbutils.fs.ls("abfss://currency@jcy203datalake.dfs.core.windows.net/raw")
today = datetime.today().strftime("%Y%m%d")
raw_currency = spark.read.json(f'abfss://currency@jcy203datalake.dfs.core.windows.net/raw/{today}')

#Transformación
currency_names = raw_currency.select("conversion_rates").schema[0].dataType.names
stack_expr = f"stack({len(currency_names)}, " + ", ".join(
    [f"'{m}', CAST(conversion_rates.`{m}` AS DOUBLE)" for m in currency_names]
) + ") as (Currency, Rate)"

currency_rate = raw_currency.select(expr(stack_expr))

#No necesito usar distint porque se que el json no tiene monedas duplicadas.
distinct_currencies = []
for row in currency_rate.select("Currency").collect():
    distinct_currencies.append(row["Currency"])

#Paso a Curated
for currency in distinct_currencies:
   currency_rate.filter(col('Currency') == currency).write.mode("overwrite").csv(f"abfss://currency@jcy203datalake.dfs.core.windows.net/curated/{currency}/{today}")
